# Note

#### 1. use kernel_initializer = 'ones' instead of 'zeros' -> the gradients do not flow through the ReLu layer

In [1]:
import numpy as np

In [2]:
# create dummy files
np.random.seed(42)
EMBEDDING_SIZE = 15
X = np.random.randint(2, size=(1000,100))
X_val = np.random.randint(2, size=(1000,100))
y = np.random.randint(2, size=1000)
y_val = np.random.randint(2, size=1000)

In [3]:
X.shape

(1000, 100)

In [4]:
y.shape

(1000,)

In [265]:
from keras.layers import Input, Dense, TimeDistributed, Embedding
from keras.layers import Concatenate, Reshape, Lambda, Multiply, multiply, concatenate
from keras.models import Model
from keras import backend as K
import tensorflow as tf

In [6]:
y_tanh = []
y_tanh_val = []

for i in y:
    if i==0:
        y_tanh.append(-1)
    else:
        y_tanh.append(1)


for i in y_val:
    if i==0:
        y_tanh_val.append(-1)
    else:
        y_tanh_val.append(1)
        
y_tanh = np.asarray(y_tanh)  
y_tanh_val = np.asarray(y_tanh_val)

In [198]:
# make first model

def build_base_model(input_shape):
    input_layer = Input(shape=(input_shape,))
    tanh_output = Dense(1, activation='tanh', name='tanh_output')(input_layer)
    
    model = Model(inputs=input_layer, outputs=tanh_output)
    model.summary()
    return model

base_model = build_base_model(X.shape[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_101 (InputLayer)       (None, 100)               0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 101       
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Combined model

combined_input_layer = Input(shape=(X.shape[1],))

ht_1 = Input(shape=(1,))
ht_2 = Input(shape=(1,))

label_layer = base_model(combined_input_layer)

dense_1 = Dense(1, activation='relu')(Multiply()([ht_1, label_layer]))
dense_2 = Dense(1, activation='relu')(Multiply()([ht_2, label_layer]))

concat = Concatenate()([dense_1, dense_2])

output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_1, ht_2], outputs=output_layer)
combined_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            101         input_2[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
multiply_1

In [9]:
base_model.compile(loss='mse',
            optimizer='adam',
            metrics=['acc'])

In [10]:
combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [11]:
# make second model
# first, make the vector to see if it is appears 

# we have two terms (human-terms) for now. c1 == pos word, c2 == neg word
# 1st column gives the feature index where that particular human_terms located
# 2nd column gives whether it is positive/negative word
term = [[1,1],[2,-1]]
term = np.asarray(term)

#assume X is a document vector [1,n] where n is the number of terms

def np_present_by_term(X, term):
    np_vector = np.zeros([term.shape[0],1])
    
    for i,(feature_index,np_term) in enumerate(term):
        if X[feature_index] == 1:
            np_vector[i] = np_term
    
    return np_vector

# Assuming 'term' is a [1X2] vector
def np_present_by_doc(X, term):
    np_vector = np.zeros([X.shape[0],1])
    
    for i, x in enumerate(X):
        if x[term[0]] == 1:
            np_vector[i] = term[1]
    
    return np_vector

In [12]:
ht_1_input = np_present_by_doc(X, term[0])
ht_2_input = np_present_by_doc(X, term[1])

ht_1_input_val = np_present_by_doc(X_val, term[0])
ht_2_input_val = np_present_by_doc(X_val, term[1])

In [13]:
base_model.fit(X, y_tanh, validation_data=(X_val, y_tanh_val), epochs=10, verbose=1)

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 196us/step - loss: 1.2646 - acc: 0.2510 - val_loss: 1.2738 - val_acc: 0.2300
Epoch 2/10
1000/1000 [==============================] - 0s 37us/step - loss: 1.2296 - acc: 0.2520 - val_loss: 1.2625 - val_acc: 0.2260
Epoch 3/10
1000/1000 [==============================] - 0s 30us/step - loss: 1.2123 - acc: 0.2490 - val_loss: 1.2414 - val_acc: 0.2110
Epoch 4/10
1000/1000 [==============================] - 0s 40us/step - loss: 1.1942 - acc: 0.2370 - val_loss: 1.2582 - val_acc: 0.2170
Epoch 5/10
1000/1000 [==============================] - 0s 35us/step - loss: 1.1794 - acc: 0.2440 - val_loss: 1.2443 - val_acc: 0.2110
Epoch 6/10
1000/1000 [==============================] - 0s 38us/step - loss: 1.1620 - acc: 0.2400 - val_loss: 1.2324 - val_acc: 0.1980
Epoch 7/10
1000/1000 [==============================] - 0s 39us/step - loss: 1.1470 - acc: 0.2280 - val_loss: 1.2268 - val_acc: 0.1960
Epoch 

In [14]:
combined_model.fit([X, ht_1_input, ht_2_input], y, 
                   validation_data=([X_val, ht_1_input_val, ht_2_input_val], y_val), 
                   epochs=10, verbose=1)

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 271us/step - loss: 0.2499 - acc: 0.5060 - val_loss: 0.2500 - val_acc: 0.4820
Epoch 2/10
1000/1000 [==============================] - 0s 50us/step - loss: 0.2496 - acc: 0.5440 - val_loss: 0.2502 - val_acc: 0.4800
Epoch 3/10
1000/1000 [==============================] - 0s 49us/step - loss: 0.2493 - acc: 0.5380 - val_loss: 0.2505 - val_acc: 0.4700
Epoch 4/10
1000/1000 [==============================] - 0s 50us/step - loss: 0.2491 - acc: 0.5410 - val_loss: 0.2507 - val_acc: 0.4730
Epoch 5/10
1000/1000 [==============================] - 0s 50us/step - loss: 0.2489 - acc: 0.5330 - val_loss: 0.2510 - val_acc: 0.4700
Epoch 6/10
1000/1000 [==============================] - 0s 49us/step - loss: 0.2485 - acc: 0.5410 - val_loss: 0.2511 - val_acc: 0.4730
Epoch 7/10
1000/1000 [==============================] - 0s 49us/step - loss: 0.2482 - acc: 0.5440 - val_loss: 0.2513 - val_acc: 0.4710
Epoch 

### Simple Domain Test

In [15]:
# Generate 16 documents
# n1 = negative word with importance 1
# p2 = positive word with importance 2
# p3 = positive word with importance 3
# n4 = negative word with importance 4
# movie = neutral word with importance 0

word_list = ['movie', 'bad', 'good', 'amazing', 'awful']
word_weight = np.asarray([0, -1, 2, 3, -4], dtype='int32')
word_weight.shape

(5,)

In [137]:
from itertools import combinations

num = 16
data = np.zeros([num, len(word_list)])
index=0
label = []

for i in range(5):
    # get the combination
    perm = combinations([1,2,3,4], i)
    
    for j in list(perm):
        # always put neutral word
        data[index, 0] = 1
        
        # if its only 1 terms
        if i==1:
            data[index,j] = 1
        else:
            for k in j:
                data[index, k] = 1
        
        # calculate weights
        
        weights = data[index,:] * word_weight
        label.append(np.sign(np.sum(weights)))
        
        index+=1
        
label = np.asarray(label)

In [138]:
np_label = (label == 1).astype('int32')
np_label

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0])

In [139]:
label

array([ 0., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,  1., -1.,
       -1.,  1.,  0.])

In [140]:
data.shape[0]

16

In [216]:
# build the network

base_model = build_base_model(data.shape[1])

# build second model

# Combined model

combined_input_layer = Input(shape=(data.shape[1],))

# build the hard coded weight for human terms
ht_1 = Input(shape=(1,))
ht_2 = Input(shape=(1,))
ht_3 = Input(shape=(1,))
ht_4 = Input(shape=(1,))

label_layer = base_model(combined_input_layer)

# multiply and pass it into relu
dense_1 = Dense(1, activation='relu', name='dense_1', use_bias=False, kernel_initializer='ones')(Multiply(name='multiply_1')([ht_1, label_layer]))
dense_2 = Dense(1, activation='relu', name='dense_2', use_bias=False, kernel_initializer='ones')(Multiply(name='multiply_2')([ht_2, label_layer]))
dense_3 = Dense(1, activation='relu', name='dense_3', use_bias=False, kernel_initializer='ones')(Multiply(name='multiply_3')([ht_3, label_layer]))
dense_4 = Dense(1, activation='relu', name='dense_4', use_bias=False, kernel_initializer='ones')(Multiply(name='multiply_4')([ht_4, label_layer]))

# concat all the result
concat = Concatenate(name='concatenate')([dense_1, dense_2, dense_3, dense_4])

output_layer = Dense(1, activation='sigmoid')(concat)


combined_model = Model(inputs=[combined_input_layer, ht_1, ht_2, ht_3, ht_4], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_114 (InputLayer)       (None, 5)                 0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_115 (InputLayer)          (None, 5)            0                                            
__________________________________________________________________________________________________
input_116 (InputLayer)          (None, 1)            0                                            
_________________________________________________

In [217]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [218]:
def np_present_by_doc(X, index, weight):
    np_vector = np.zeros([X.shape[0],1])
    
    for i, x in enumerate(X):
        if x[index] == 1:
            np_vector[i] = np.sign(weight)
    
    return np_vector

In [219]:
# I need to generate the ht's

ht_1_input = np_present_by_doc(data, 1, word_weight[1])
ht_2_input = np_present_by_doc(data, 2, word_weight[2])
ht_3_input = np_present_by_doc(data, 3, word_weight[3])
ht_4_input = np_present_by_doc(data, 4, word_weight[4])

In [305]:
ht_1_input.shape

(16, 1)

In [306]:
ht_input = np.hstack([ht_1_input, ht_2_input, ht_3_input, ht_4_input])

In [308]:
ht_input

array([[ 0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0., -1.],
       [-1.,  1.,  0.,  0.],
       [-1.,  0.,  1.,  0.],
       [-1.,  0.,  0., -1.],
       [ 0.,  1.,  1.,  0.],
       [ 0.,  1.,  0., -1.],
       [ 0.,  0.,  1., -1.],
       [-1.,  1.,  1.,  0.],
       [-1.,  1.,  0., -1.],
       [-1.,  0.,  1., -1.],
       [ 0.,  1.,  1., -1.],
       [-1.,  1.,  1., -1.]])

In [220]:
for i in range(len(ht_1_input)):
    print(data[i], ht_4_input[i])

[1. 0. 0. 0. 0.] [0.]
[1. 1. 0. 0. 0.] [0.]
[1. 0. 1. 0. 0.] [0.]
[1. 0. 0. 1. 0.] [0.]
[1. 0. 0. 0. 1.] [-1.]
[1. 1. 1. 0. 0.] [0.]
[1. 1. 0. 1. 0.] [0.]
[1. 1. 0. 0. 1.] [-1.]
[1. 0. 1. 1. 0.] [0.]
[1. 0. 1. 0. 1.] [-1.]
[1. 0. 0. 1. 1.] [-1.]
[1. 1. 1. 1. 0.] [0.]
[1. 1. 1. 0. 1.] [-1.]
[1. 1. 0. 1. 1.] [-1.]
[1. 0. 1. 1. 1.] [-1.]
[1. 1. 1. 1. 1.] [-1.]


In [221]:
# # adjust y to -> [-1, 1]
# label_tanh = []
# for i in label:
#     if i==0:
#         label_tanh.append(-1)
#     else:
#         label_tanh.append(1)
# label_tanh = np.asarray(label_tanh)

In [222]:
base_model.fit(data[1:15], label[1:15], epochs=1000, batch_size=1, verbose=2)

Epoch 1/1000
 - 1s - loss: 1.0891 - acc: 0.5000
Epoch 2/1000
 - 0s - loss: 1.0570 - acc: 0.5000
Epoch 3/1000
 - 0s - loss: 1.0255 - acc: 0.5000
Epoch 4/1000
 - 0s - loss: 0.9976 - acc: 0.5000
Epoch 5/1000
 - 0s - loss: 0.9726 - acc: 0.5000
Epoch 6/1000
 - 0s - loss: 0.9427 - acc: 0.5000
Epoch 7/1000
 - 0s - loss: 0.9132 - acc: 0.5000
Epoch 8/1000
 - 0s - loss: 0.8901 - acc: 0.5000
Epoch 9/1000
 - 0s - loss: 0.8594 - acc: 0.4286
Epoch 10/1000
 - 0s - loss: 0.8358 - acc: 0.3571
Epoch 11/1000
 - 0s - loss: 0.8126 - acc: 0.3571
Epoch 12/1000
 - 0s - loss: 0.7904 - acc: 0.3571
Epoch 13/1000
 - 0s - loss: 0.7697 - acc: 0.3571
Epoch 14/1000
 - 0s - loss: 0.7462 - acc: 0.3571
Epoch 15/1000
 - 0s - loss: 0.7291 - acc: 0.3571
Epoch 16/1000
 - 0s - loss: 0.7119 - acc: 0.3571
Epoch 17/1000
 - 0s - loss: 0.6915 - acc: 0.3571
Epoch 18/1000
 - 0s - loss: 0.6787 - acc: 0.3571
Epoch 19/1000
 - 0s - loss: 0.6620 - acc: 0.3571
Epoch 20/1000
 - 0s - loss: 0.6467 - acc: 0.3571
Epoch 21/1000
 - 0s - loss: 0

 - 0s - loss: 0.2247 - acc: 0.5714
Epoch 168/1000
 - 0s - loss: 0.2237 - acc: 0.5714
Epoch 169/1000
 - 0s - loss: 0.2228 - acc: 0.5714
Epoch 170/1000
 - 0s - loss: 0.2222 - acc: 0.5714
Epoch 171/1000
 - 0s - loss: 0.2206 - acc: 0.5714
Epoch 172/1000
 - 0s - loss: 0.2202 - acc: 0.5714
Epoch 173/1000
 - 0s - loss: 0.2188 - acc: 0.5714
Epoch 174/1000
 - 0s - loss: 0.2179 - acc: 0.5714
Epoch 175/1000
 - 0s - loss: 0.2171 - acc: 0.5714
Epoch 176/1000
 - 0s - loss: 0.2158 - acc: 0.5714
Epoch 177/1000
 - 0s - loss: 0.2149 - acc: 0.5714
Epoch 178/1000
 - 0s - loss: 0.2139 - acc: 0.5714
Epoch 179/1000
 - 0s - loss: 0.2131 - acc: 0.5714
Epoch 180/1000
 - 0s - loss: 0.2121 - acc: 0.5714
Epoch 181/1000
 - 0s - loss: 0.2118 - acc: 0.5714
Epoch 182/1000
 - 0s - loss: 0.2106 - acc: 0.5714
Epoch 183/1000
 - 0s - loss: 0.2100 - acc: 0.5714
Epoch 184/1000
 - 0s - loss: 0.2089 - acc: 0.5714
Epoch 185/1000
 - 0s - loss: 0.2081 - acc: 0.5714
Epoch 186/1000
 - 0s - loss: 0.2068 - acc: 0.5714
Epoch 187/1000


 - 0s - loss: 0.1201 - acc: 0.8571
Epoch 332/1000
 - 0s - loss: 0.1200 - acc: 0.8571
Epoch 333/1000
 - 0s - loss: 0.1194 - acc: 0.8571
Epoch 334/1000
 - 0s - loss: 0.1191 - acc: 0.8571
Epoch 335/1000
 - 0s - loss: 0.1183 - acc: 0.8571
Epoch 336/1000
 - 0s - loss: 0.1181 - acc: 0.8571
Epoch 337/1000
 - 0s - loss: 0.1175 - acc: 0.8571
Epoch 338/1000
 - 0s - loss: 0.1172 - acc: 0.8571
Epoch 339/1000
 - 0s - loss: 0.1169 - acc: 0.8571
Epoch 340/1000
 - 0s - loss: 0.1165 - acc: 0.8571
Epoch 341/1000
 - 0s - loss: 0.1160 - acc: 0.8571
Epoch 342/1000
 - 0s - loss: 0.1156 - acc: 0.8571
Epoch 343/1000
 - 0s - loss: 0.1155 - acc: 0.8571
Epoch 344/1000
 - 0s - loss: 0.1149 - acc: 0.8571
Epoch 345/1000
 - 0s - loss: 0.1145 - acc: 0.8571
Epoch 346/1000
 - 0s - loss: 0.1142 - acc: 0.8571
Epoch 347/1000
 - 0s - loss: 0.1136 - acc: 0.8571
Epoch 348/1000
 - 0s - loss: 0.1134 - acc: 0.8571
Epoch 349/1000
 - 0s - loss: 0.1129 - acc: 0.8571
Epoch 350/1000
 - 0s - loss: 0.1129 - acc: 0.8571
Epoch 351/1000


Epoch 496/1000
 - 0s - loss: 0.0696 - acc: 1.0000
Epoch 497/1000
 - 0s - loss: 0.0695 - acc: 1.0000
Epoch 498/1000
 - 0s - loss: 0.0691 - acc: 1.0000
Epoch 499/1000
 - 0s - loss: 0.0689 - acc: 1.0000
Epoch 500/1000
 - 0s - loss: 0.0688 - acc: 1.0000
Epoch 501/1000
 - 0s - loss: 0.0685 - acc: 1.0000
Epoch 502/1000
 - 0s - loss: 0.0683 - acc: 1.0000
Epoch 503/1000
 - 0s - loss: 0.0680 - acc: 1.0000
Epoch 504/1000
 - 0s - loss: 0.0678 - acc: 1.0000
Epoch 505/1000
 - 0s - loss: 0.0678 - acc: 1.0000
Epoch 506/1000
 - 0s - loss: 0.0674 - acc: 1.0000
Epoch 507/1000
 - 0s - loss: 0.0674 - acc: 1.0000
Epoch 508/1000
 - 0s - loss: 0.0669 - acc: 1.0000
Epoch 509/1000
 - 0s - loss: 0.0667 - acc: 1.0000
Epoch 510/1000
 - 0s - loss: 0.0665 - acc: 1.0000
Epoch 511/1000
 - 0s - loss: 0.0663 - acc: 1.0000
Epoch 512/1000
 - 0s - loss: 0.0661 - acc: 1.0000
Epoch 513/1000
 - 0s - loss: 0.0659 - acc: 1.0000
Epoch 514/1000
 - 0s - loss: 0.0656 - acc: 1.0000
Epoch 515/1000
 - 0s - loss: 0.0655 - acc: 1.0000


Epoch 660/1000
 - 0s - loss: 0.0411 - acc: 1.0000
Epoch 661/1000
 - 0s - loss: 0.0410 - acc: 1.0000
Epoch 662/1000
 - 0s - loss: 0.0409 - acc: 1.0000
Epoch 663/1000
 - 0s - loss: 0.0407 - acc: 1.0000
Epoch 664/1000
 - 0s - loss: 0.0407 - acc: 1.0000
Epoch 665/1000
 - 0s - loss: 0.0405 - acc: 1.0000
Epoch 666/1000
 - 0s - loss: 0.0403 - acc: 1.0000
Epoch 667/1000
 - 0s - loss: 0.0403 - acc: 1.0000
Epoch 668/1000
 - 0s - loss: 0.0401 - acc: 1.0000
Epoch 669/1000
 - 0s - loss: 0.0400 - acc: 1.0000
Epoch 670/1000
 - 0s - loss: 0.0399 - acc: 1.0000
Epoch 671/1000
 - 0s - loss: 0.0397 - acc: 1.0000
Epoch 672/1000
 - 0s - loss: 0.0396 - acc: 1.0000
Epoch 673/1000
 - 0s - loss: 0.0394 - acc: 1.0000
Epoch 674/1000
 - 0s - loss: 0.0394 - acc: 1.0000
Epoch 675/1000
 - 0s - loss: 0.0393 - acc: 1.0000
Epoch 676/1000
 - 0s - loss: 0.0392 - acc: 1.0000
Epoch 677/1000
 - 0s - loss: 0.0390 - acc: 1.0000
Epoch 678/1000
 - 0s - loss: 0.0388 - acc: 1.0000
Epoch 679/1000
 - 0s - loss: 0.0387 - acc: 1.0000


Epoch 824/1000
 - 0s - loss: 0.0244 - acc: 1.0000
Epoch 825/1000
 - 0s - loss: 0.0243 - acc: 1.0000
Epoch 826/1000
 - 0s - loss: 0.0242 - acc: 1.0000
Epoch 827/1000
 - 0s - loss: 0.0241 - acc: 1.0000
Epoch 828/1000
 - 0s - loss: 0.0242 - acc: 1.0000
Epoch 829/1000
 - 0s - loss: 0.0240 - acc: 1.0000
Epoch 830/1000
 - 0s - loss: 0.0239 - acc: 1.0000
Epoch 831/1000
 - 0s - loss: 0.0239 - acc: 1.0000
Epoch 832/1000
 - 0s - loss: 0.0238 - acc: 1.0000
Epoch 833/1000
 - 0s - loss: 0.0236 - acc: 1.0000
Epoch 834/1000
 - 0s - loss: 0.0236 - acc: 1.0000
Epoch 835/1000
 - 0s - loss: 0.0237 - acc: 1.0000
Epoch 836/1000
 - 0s - loss: 0.0235 - acc: 1.0000
Epoch 837/1000
 - 0s - loss: 0.0234 - acc: 1.0000
Epoch 838/1000
 - 0s - loss: 0.0234 - acc: 1.0000
Epoch 839/1000
 - 0s - loss: 0.0232 - acc: 1.0000
Epoch 840/1000
 - 0s - loss: 0.0232 - acc: 1.0000
Epoch 841/1000
 - 0s - loss: 0.0231 - acc: 1.0000
Epoch 842/1000
 - 0s - loss: 0.0230 - acc: 1.0000
Epoch 843/1000
 - 0s - loss: 0.0229 - acc: 1.0000


Epoch 988/1000
 - 0s - loss: 0.0144 - acc: 1.0000
Epoch 989/1000
 - 0s - loss: 0.0144 - acc: 1.0000
Epoch 990/1000
 - 0s - loss: 0.0144 - acc: 1.0000
Epoch 991/1000
 - 0s - loss: 0.0143 - acc: 1.0000
Epoch 992/1000
 - 0s - loss: 0.0142 - acc: 1.0000
Epoch 993/1000
 - 0s - loss: 0.0142 - acc: 1.0000
Epoch 994/1000
 - 0s - loss: 0.0142 - acc: 1.0000
Epoch 995/1000
 - 0s - loss: 0.0141 - acc: 1.0000
Epoch 996/1000
 - 0s - loss: 0.0141 - acc: 1.0000
Epoch 997/1000
 - 0s - loss: 0.0140 - acc: 1.0000
Epoch 998/1000
 - 0s - loss: 0.0140 - acc: 1.0000
Epoch 999/1000
 - 0s - loss: 0.0139 - acc: 1.0000
Epoch 1000/1000
 - 0s - loss: 0.0139 - acc: 1.0000


In [223]:
y = base_model.predict(data[1:15])
for i in range(len(y)):
    print(data[i+1], label[i+1], y[i])

[1. 1. 0. 0. 0.] -1.0 [-0.8072424]
[1. 0. 1. 0. 0.] 1.0 [0.92721075]
[1. 0. 0. 1. 0.] 1.0 [0.92723894]
[1. 0. 0. 0. 1.] -1.0 [-0.9984307]
[1. 1. 1. 0. 0.] 1.0 [0.84543383]
[1. 1. 0. 1. 0.] 1.0 [0.8454914]
[1. 1. 0. 0. 1.] -1.0 [-0.99929196]
[1. 0. 1. 1. 0.] 1.0 [0.9993257]
[1. 0. 1. 0. 1.] -1.0 [-0.83840513]
[1. 0. 0. 1. 1.] -1.0 [-0.83834535]
[1. 1. 1. 1. 0.] 1.0 [0.9985053]
[1. 1. 1. 0. 1.] -1.0 [-0.92374706]
[1. 1. 0. 1. 1.] -1.0 [-0.92371744]
[1. 0. 1. 1. 1.] 1.0 [0.8155579]


In [224]:
combined_model.fit([data[1:15], ht_1_input[1:15], ht_2_input[1:15], ht_3_input[1:15], ht_4_input[1:15]], 
                   np_label[1:15], 
                   epochs=500,
                   batch_size=1,
                   verbose=2)

Epoch 1/500
 - 1s - loss: 0.2342 - acc: 0.5714
Epoch 2/500
 - 0s - loss: 0.2303 - acc: 0.5714
Epoch 3/500
 - 0s - loss: 0.2263 - acc: 0.5714
Epoch 4/500
 - 0s - loss: 0.2223 - acc: 0.5714
Epoch 5/500
 - 0s - loss: 0.2189 - acc: 0.5714
Epoch 6/500
 - 0s - loss: 0.2148 - acc: 0.5714
Epoch 7/500
 - 0s - loss: 0.2115 - acc: 0.5714
Epoch 8/500
 - 0s - loss: 0.2078 - acc: 0.5714
Epoch 9/500
 - 0s - loss: 0.2043 - acc: 0.5714
Epoch 10/500
 - 0s - loss: 0.2010 - acc: 0.5714
Epoch 11/500
 - 0s - loss: 0.1976 - acc: 0.5714
Epoch 12/500
 - 0s - loss: 0.1944 - acc: 0.6429
Epoch 13/500
 - 0s - loss: 0.1909 - acc: 0.7857
Epoch 14/500
 - 0s - loss: 0.1879 - acc: 0.7857
Epoch 15/500
 - 0s - loss: 0.1847 - acc: 0.7857
Epoch 16/500
 - 0s - loss: 0.1818 - acc: 0.7857
Epoch 17/500
 - 0s - loss: 0.1789 - acc: 0.7857
Epoch 18/500
 - 0s - loss: 0.1759 - acc: 0.7857
Epoch 19/500
 - 0s - loss: 0.1732 - acc: 0.7857
Epoch 20/500
 - 0s - loss: 0.1703 - acc: 0.7857
Epoch 21/500
 - 0s - loss: 0.1676 - acc: 0.7857
E

Epoch 171/500
 - 0s - loss: 0.0345 - acc: 1.0000
Epoch 172/500
 - 0s - loss: 0.0342 - acc: 1.0000
Epoch 173/500
 - 0s - loss: 0.0339 - acc: 1.0000
Epoch 174/500
 - 0s - loss: 0.0336 - acc: 1.0000
Epoch 175/500
 - 0s - loss: 0.0332 - acc: 1.0000
Epoch 176/500
 - 0s - loss: 0.0330 - acc: 1.0000
Epoch 177/500
 - 0s - loss: 0.0326 - acc: 1.0000
Epoch 178/500
 - 0s - loss: 0.0323 - acc: 1.0000
Epoch 179/500
 - 0s - loss: 0.0321 - acc: 1.0000
Epoch 180/500
 - 0s - loss: 0.0317 - acc: 1.0000
Epoch 181/500
 - 0s - loss: 0.0313 - acc: 1.0000
Epoch 182/500
 - 0s - loss: 0.0311 - acc: 1.0000
Epoch 183/500
 - 0s - loss: 0.0307 - acc: 1.0000
Epoch 184/500
 - 0s - loss: 0.0305 - acc: 1.0000
Epoch 185/500
 - 0s - loss: 0.0301 - acc: 1.0000
Epoch 186/500
 - 0s - loss: 0.0298 - acc: 1.0000
Epoch 187/500
 - 0s - loss: 0.0295 - acc: 1.0000
Epoch 188/500
 - 0s - loss: 0.0292 - acc: 1.0000
Epoch 189/500
 - 0s - loss: 0.0289 - acc: 1.0000
Epoch 190/500
 - 0s - loss: 0.0286 - acc: 1.0000
Epoch 191/500
 - 0s 

Epoch 339/500
 - 0s - loss: 0.0049 - acc: 1.0000
Epoch 340/500
 - 0s - loss: 0.0048 - acc: 1.0000
Epoch 341/500
 - 0s - loss: 0.0047 - acc: 1.0000
Epoch 342/500
 - 0s - loss: 0.0047 - acc: 1.0000
Epoch 343/500
 - 0s - loss: 0.0046 - acc: 1.0000
Epoch 344/500
 - 0s - loss: 0.0046 - acc: 1.0000
Epoch 345/500
 - 0s - loss: 0.0045 - acc: 1.0000
Epoch 346/500
 - 0s - loss: 0.0045 - acc: 1.0000
Epoch 347/500
 - 0s - loss: 0.0044 - acc: 1.0000
Epoch 348/500
 - 0s - loss: 0.0044 - acc: 1.0000
Epoch 349/500
 - 0s - loss: 0.0043 - acc: 1.0000
Epoch 350/500
 - 0s - loss: 0.0043 - acc: 1.0000
Epoch 351/500
 - 0s - loss: 0.0042 - acc: 1.0000
Epoch 352/500
 - 0s - loss: 0.0042 - acc: 1.0000
Epoch 353/500
 - 0s - loss: 0.0041 - acc: 1.0000
Epoch 354/500
 - 0s - loss: 0.0041 - acc: 1.0000
Epoch 355/500
 - 0s - loss: 0.0041 - acc: 1.0000
Epoch 356/500
 - 0s - loss: 0.0040 - acc: 1.0000
Epoch 357/500
 - 0s - loss: 0.0040 - acc: 1.0000
Epoch 358/500
 - 0s - loss: 0.0039 - acc: 1.0000
Epoch 359/500
 - 0s 

In [225]:
net_weight = combined_model.get_weights()
net_weight

[array([[-1.0029436],
        [-0.601707 ],
        [ 3.1666331],
        [ 2.96085  ],
        [-3.4908283]], dtype=float32),
 array([-0.06363073], dtype=float32),
 array([[2.3106627]], dtype=float32),
 array([[2.117043]], dtype=float32),
 array([[2.5342627]], dtype=float32),
 array([[2.38485]], dtype=float32),
 array([[-1.3162277],
        [ 1.3323443],
        [ 1.4503046],
        [-2.0237808]], dtype=float32),
 array([0.36625943], dtype=float32)]

In [316]:
def report():
    for i in range(1,15):
        bm = base_model.predict(np.reshape(data[i], (1,5)))
        
        cm = combined_model.predict([np.reshape(data[i], (1,5)), 
                                np.reshape(ht_input[i], (1,4))])
        
#         document_output = 'multiply'
#         document_predict = Model(inputs=combined_model.input,
#                                      outputs=combined_model.get_layer(document_output).output)
#         doc_output = document_predict.predict([np.reshape(data[i], (1,5)), 
#                                       ht_1_input[i], 
#                                       ht_2_input[i], 
#                                       ht_3_input[i], 
#                                       ht_4_input[i]])
        
        layer_name = 'concatenate'
        concat_after_relu = Model(inputs=combined_model.input,
                                     outputs=combined_model.get_layer(layer_name).output)
        concat_output = concat_after_relu.predict([np.reshape(data[i], (1,5)), 
                                     np.reshape(ht_input[i], (1,4))])
        
        print(data[i], '\t', np_label[i], '\t', bm.flatten(), '\t', cm.flatten(), '\t',concat_output.flatten())
    

In [317]:
ht_input.shape

(16, 4)

In [318]:
report()

ValueError: No such layer: concatenate

In [190]:
word_weight

array([ 0, -1,  2,  3, -4])

In [191]:
from keras.models import load_model

# base_model.save('overfit_base_model.h5')
# combined_model.save('overfit_combined_model.h5')

In [33]:
# bm = base_model.predict(np.reshape(data[test_idx], (1,5)))
# print(bm)

# before jointly trained.
# 0.7995629

### IMDB network Model

In [319]:
# build the combined model
# Combined model
human_terms_len = 4

base_model = build_base_model(data.shape[1])

combined_input_layer = Input(shape=(data.shape[1],))

# build the hard coded weight for human terms
ht_input_layer = Input(shape=(human_terms_len,))

split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

# get the document prediction
label_layer = base_model(combined_input_layer)

# multiply and pass it into relu
# initialize relu layer

relu_layer = Dense(1, activation='relu', name='relu_layer', use_bias=False, kernel_initializer='ones')

# stack the multiply layer
dense_layer = []
for i in range(human_terms_len):
    dense_layer.append(relu_layer(Multiply()([split[i], label_layer])))

# concat all the result   
concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

# pass it to sigmoid layer
output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_252 (InputLayer)       (None, 5)                 0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_254 (InputLayer)          (None, 4)            0                                            
__________________________________________________________________________________________________
input_253 (InputLayer)          (None, 5)            0                                            
_________________________________________________

In [320]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [321]:
base_model.fit(data[1:15], label[1:15], epochs=1000, batch_size=1, verbose=2)

Epoch 1/1000
 - 1s - loss: 1.7918 - acc: 0.4286
Epoch 2/1000
 - 0s - loss: 1.7855 - acc: 0.4286
Epoch 3/1000
 - 0s - loss: 1.7799 - acc: 0.4286
Epoch 4/1000
 - 0s - loss: 1.7737 - acc: 0.4286
Epoch 5/1000
 - 0s - loss: 1.7677 - acc: 0.4286
Epoch 6/1000
 - 0s - loss: 1.7608 - acc: 0.4286
Epoch 7/1000
 - 0s - loss: 1.7537 - acc: 0.4286
Epoch 8/1000
 - 0s - loss: 1.7477 - acc: 0.4286
Epoch 9/1000
 - 0s - loss: 1.7414 - acc: 0.4286
Epoch 10/1000
 - 0s - loss: 1.7315 - acc: 0.4286
Epoch 11/1000
 - 0s - loss: 1.7235 - acc: 0.4286
Epoch 12/1000
 - 0s - loss: 1.7168 - acc: 0.4286
Epoch 13/1000
 - 0s - loss: 1.7071 - acc: 0.4286
Epoch 14/1000
 - 0s - loss: 1.6998 - acc: 0.4286
Epoch 15/1000
 - 0s - loss: 1.6893 - acc: 0.4286
Epoch 16/1000
 - 0s - loss: 1.6802 - acc: 0.4286
Epoch 17/1000
 - 0s - loss: 1.6708 - acc: 0.4286
Epoch 18/1000
 - 0s - loss: 1.6592 - acc: 0.4286
Epoch 19/1000
 - 0s - loss: 1.6498 - acc: 0.4286
Epoch 20/1000
 - 0s - loss: 1.6385 - acc: 0.4286
Epoch 21/1000
 - 0s - loss: 1

 - 0s - loss: 0.4038 - acc: 0.4286
Epoch 168/1000
 - 0s - loss: 0.4026 - acc: 0.4286
Epoch 169/1000
 - 0s - loss: 0.3991 - acc: 0.4286
Epoch 170/1000
 - 0s - loss: 0.3963 - acc: 0.4286
Epoch 171/1000
 - 0s - loss: 0.3942 - acc: 0.4286
Epoch 172/1000
 - 0s - loss: 0.3911 - acc: 0.4286
Epoch 173/1000
 - 0s - loss: 0.3895 - acc: 0.4286
Epoch 174/1000
 - 0s - loss: 0.3856 - acc: 0.4286
Epoch 175/1000
 - 0s - loss: 0.3838 - acc: 0.4286
Epoch 176/1000
 - 0s - loss: 0.3814 - acc: 0.4286
Epoch 177/1000
 - 0s - loss: 0.3791 - acc: 0.4286
Epoch 178/1000
 - 0s - loss: 0.3767 - acc: 0.4286
Epoch 179/1000
 - 0s - loss: 0.3745 - acc: 0.4286
Epoch 180/1000
 - 0s - loss: 0.3725 - acc: 0.4286
Epoch 181/1000
 - 0s - loss: 0.3706 - acc: 0.4286
Epoch 182/1000
 - 0s - loss: 0.3677 - acc: 0.4286
Epoch 183/1000
 - 0s - loss: 0.3654 - acc: 0.4286
Epoch 184/1000
 - 0s - loss: 0.3635 - acc: 0.4286
Epoch 185/1000
 - 0s - loss: 0.3611 - acc: 0.4286
Epoch 186/1000
 - 0s - loss: 0.3594 - acc: 0.4286
Epoch 187/1000


 - 0s - loss: 0.1934 - acc: 0.5714
Epoch 332/1000
 - 0s - loss: 0.1930 - acc: 0.5714
Epoch 333/1000
 - 0s - loss: 0.1921 - acc: 0.5714
Epoch 334/1000
 - 0s - loss: 0.1915 - acc: 0.5714
Epoch 335/1000
 - 0s - loss: 0.1905 - acc: 0.5714
Epoch 336/1000
 - 0s - loss: 0.1905 - acc: 0.5714
Epoch 337/1000
 - 0s - loss: 0.1899 - acc: 0.5714
Epoch 338/1000
 - 0s - loss: 0.1886 - acc: 0.5714
Epoch 339/1000
 - 0s - loss: 0.1882 - acc: 0.5714
Epoch 340/1000
 - 0s - loss: 0.1878 - acc: 0.5714
Epoch 341/1000
 - 0s - loss: 0.1868 - acc: 0.5714
Epoch 342/1000
 - 0s - loss: 0.1865 - acc: 0.5714
Epoch 343/1000
 - 0s - loss: 0.1855 - acc: 0.5714
Epoch 344/1000
 - 0s - loss: 0.1851 - acc: 0.5714
Epoch 345/1000
 - 0s - loss: 0.1849 - acc: 0.5714
Epoch 346/1000
 - 0s - loss: 0.1845 - acc: 0.5714
Epoch 347/1000
 - 0s - loss: 0.1831 - acc: 0.5714
Epoch 348/1000
 - 0s - loss: 0.1825 - acc: 0.5714
Epoch 349/1000
 - 0s - loss: 0.1819 - acc: 0.5714
Epoch 350/1000
 - 0s - loss: 0.1815 - acc: 0.5714
Epoch 351/1000


 - 0s - loss: 0.1142 - acc: 0.8571
Epoch 496/1000
 - 0s - loss: 0.1139 - acc: 0.8571
Epoch 497/1000
 - 0s - loss: 0.1134 - acc: 0.8571
Epoch 498/1000
 - 0s - loss: 0.1133 - acc: 0.8571
Epoch 499/1000
 - 0s - loss: 0.1133 - acc: 0.8571
Epoch 500/1000
 - 0s - loss: 0.1124 - acc: 0.8571
Epoch 501/1000
 - 0s - loss: 0.1121 - acc: 0.8571
Epoch 502/1000
 - 0s - loss: 0.1121 - acc: 0.8571
Epoch 503/1000
 - 0s - loss: 0.1114 - acc: 0.8571
Epoch 504/1000
 - 0s - loss: 0.1110 - acc: 0.8571
Epoch 505/1000
 - 0s - loss: 0.1108 - acc: 0.8571
Epoch 506/1000
 - 0s - loss: 0.1106 - acc: 0.8571
Epoch 507/1000
 - 0s - loss: 0.1101 - acc: 0.8571
Epoch 508/1000
 - 0s - loss: 0.1098 - acc: 0.8571
Epoch 509/1000
 - 0s - loss: 0.1094 - acc: 0.8571
Epoch 510/1000
 - 0s - loss: 0.1094 - acc: 0.8571
Epoch 511/1000
 - 0s - loss: 0.1087 - acc: 0.8571
Epoch 512/1000
 - 0s - loss: 0.1085 - acc: 0.8571
Epoch 513/1000
 - 0s - loss: 0.1080 - acc: 0.8571
Epoch 514/1000
 - 0s - loss: 0.1077 - acc: 0.8571
Epoch 515/1000


 - 0s - loss: 0.0688 - acc: 1.0000
Epoch 660/1000
 - 0s - loss: 0.0689 - acc: 1.0000
Epoch 661/1000
 - 0s - loss: 0.0686 - acc: 1.0000
Epoch 662/1000
 - 0s - loss: 0.0685 - acc: 1.0000
Epoch 663/1000
 - 0s - loss: 0.0681 - acc: 1.0000
Epoch 664/1000
 - 0s - loss: 0.0678 - acc: 1.0000
Epoch 665/1000
 - 0s - loss: 0.0677 - acc: 1.0000
Epoch 666/1000
 - 0s - loss: 0.0674 - acc: 1.0000
Epoch 667/1000
 - 0s - loss: 0.0672 - acc: 1.0000
Epoch 668/1000
 - 0s - loss: 0.0672 - acc: 1.0000
Epoch 669/1000
 - 0s - loss: 0.0670 - acc: 1.0000
Epoch 670/1000
 - 0s - loss: 0.0667 - acc: 1.0000
Epoch 671/1000
 - 0s - loss: 0.0665 - acc: 1.0000
Epoch 672/1000
 - 0s - loss: 0.0663 - acc: 1.0000
Epoch 673/1000
 - 0s - loss: 0.0663 - acc: 1.0000
Epoch 674/1000
 - 0s - loss: 0.0658 - acc: 1.0000
Epoch 675/1000
 - 0s - loss: 0.0658 - acc: 1.0000
Epoch 676/1000
 - 0s - loss: 0.0658 - acc: 1.0000
Epoch 677/1000
 - 0s - loss: 0.0652 - acc: 1.0000
Epoch 678/1000
 - 0s - loss: 0.0650 - acc: 1.0000
Epoch 679/1000


Epoch 824/1000
 - 0s - loss: 0.0415 - acc: 1.0000
Epoch 825/1000
 - 0s - loss: 0.0414 - acc: 1.0000
Epoch 826/1000
 - 0s - loss: 0.0412 - acc: 1.0000
Epoch 827/1000
 - 0s - loss: 0.0410 - acc: 1.0000
Epoch 828/1000
 - 0s - loss: 0.0410 - acc: 1.0000
Epoch 829/1000
 - 0s - loss: 0.0408 - acc: 1.0000
Epoch 830/1000
 - 0s - loss: 0.0408 - acc: 1.0000
Epoch 831/1000
 - 0s - loss: 0.0406 - acc: 1.0000
Epoch 832/1000
 - 0s - loss: 0.0404 - acc: 1.0000
Epoch 833/1000
 - 0s - loss: 0.0404 - acc: 1.0000
Epoch 834/1000
 - 0s - loss: 0.0402 - acc: 1.0000
Epoch 835/1000
 - 0s - loss: 0.0400 - acc: 1.0000
Epoch 836/1000
 - 0s - loss: 0.0399 - acc: 1.0000
Epoch 837/1000
 - 0s - loss: 0.0398 - acc: 1.0000
Epoch 838/1000
 - 0s - loss: 0.0397 - acc: 1.0000
Epoch 839/1000
 - 0s - loss: 0.0396 - acc: 1.0000
Epoch 840/1000
 - 0s - loss: 0.0395 - acc: 1.0000
Epoch 841/1000
 - 0s - loss: 0.0393 - acc: 1.0000
Epoch 842/1000
 - 0s - loss: 0.0393 - acc: 1.0000
Epoch 843/1000
 - 0s - loss: 0.0391 - acc: 1.0000


Epoch 988/1000
 - 0s - loss: 0.0249 - acc: 1.0000
Epoch 989/1000
 - 0s - loss: 0.0248 - acc: 1.0000
Epoch 990/1000
 - 0s - loss: 0.0247 - acc: 1.0000
Epoch 991/1000
 - 0s - loss: 0.0247 - acc: 1.0000
Epoch 992/1000
 - 0s - loss: 0.0245 - acc: 1.0000
Epoch 993/1000
 - 0s - loss: 0.0245 - acc: 1.0000
Epoch 994/1000
 - 0s - loss: 0.0244 - acc: 1.0000
Epoch 995/1000
 - 0s - loss: 0.0244 - acc: 1.0000
Epoch 996/1000
 - 0s - loss: 0.0244 - acc: 1.0000
Epoch 997/1000
 - 0s - loss: 0.0242 - acc: 1.0000
Epoch 998/1000
 - 0s - loss: 0.0241 - acc: 1.0000
Epoch 999/1000
 - 0s - loss: 0.0240 - acc: 1.0000
Epoch 1000/1000
 - 0s - loss: 0.0240 - acc: 1.0000


In [322]:
combined_model.fit([data[1:15], ht_input[1:15]], 
                   np_label[1:15], 
                   epochs=500,
                   batch_size=1,
                   verbose=2)

Epoch 1/500
 - 1s - loss: 0.3705 - acc: 0.1429
Epoch 2/500
 - 0s - loss: 0.3664 - acc: 0.1429
Epoch 3/500
 - 0s - loss: 0.3621 - acc: 0.1429
Epoch 4/500
 - 0s - loss: 0.3581 - acc: 0.1429
Epoch 5/500
 - 0s - loss: 0.3540 - acc: 0.1429
Epoch 6/500
 - 0s - loss: 0.3504 - acc: 0.1429
Epoch 7/500
 - 0s - loss: 0.3459 - acc: 0.1429
Epoch 8/500
 - 0s - loss: 0.3424 - acc: 0.1429
Epoch 9/500
 - 0s - loss: 0.3385 - acc: 0.2143
Epoch 10/500
 - 0s - loss: 0.3347 - acc: 0.3571
Epoch 11/500
 - 0s - loss: 0.3313 - acc: 0.3571
Epoch 12/500
 - 0s - loss: 0.3274 - acc: 0.3571
Epoch 13/500
 - 0s - loss: 0.3244 - acc: 0.3571
Epoch 14/500
 - 0s - loss: 0.3206 - acc: 0.3571
Epoch 15/500
 - 0s - loss: 0.3172 - acc: 0.3571
Epoch 16/500
 - 0s - loss: 0.3139 - acc: 0.3571
Epoch 17/500
 - 0s - loss: 0.3107 - acc: 0.3571
Epoch 18/500
 - 0s - loss: 0.3074 - acc: 0.3571
Epoch 19/500
 - 0s - loss: 0.3047 - acc: 0.3571
Epoch 20/500
 - 0s - loss: 0.3011 - acc: 0.3571
Epoch 21/500
 - 0s - loss: 0.2983 - acc: 0.3571
E

Epoch 171/500
 - 0s - loss: 0.0909 - acc: 0.8571
Epoch 172/500
 - 0s - loss: 0.0894 - acc: 0.8571
Epoch 173/500
 - 0s - loss: 0.0881 - acc: 0.8571
Epoch 174/500
 - 0s - loss: 0.0867 - acc: 0.8571
Epoch 175/500
 - 0s - loss: 0.0855 - acc: 0.8571
Epoch 176/500
 - 0s - loss: 0.0843 - acc: 0.8571
Epoch 177/500
 - 0s - loss: 0.0830 - acc: 0.8571
Epoch 178/500
 - 0s - loss: 0.0817 - acc: 0.8571
Epoch 179/500
 - 0s - loss: 0.0806 - acc: 0.8571
Epoch 180/500
 - 0s - loss: 0.0794 - acc: 0.8571
Epoch 181/500
 - 0s - loss: 0.0783 - acc: 0.8571
Epoch 182/500
 - 0s - loss: 0.0771 - acc: 0.8571
Epoch 183/500
 - 0s - loss: 0.0761 - acc: 0.8571
Epoch 184/500
 - 0s - loss: 0.0749 - acc: 0.8571
Epoch 185/500
 - 0s - loss: 0.0738 - acc: 0.8571
Epoch 186/500
 - 0s - loss: 0.0729 - acc: 0.8571
Epoch 187/500
 - 0s - loss: 0.0718 - acc: 0.8571
Epoch 188/500
 - 0s - loss: 0.0708 - acc: 0.9286
Epoch 189/500
 - 0s - loss: 0.0697 - acc: 1.0000
Epoch 190/500
 - 0s - loss: 0.0688 - acc: 1.0000
Epoch 191/500
 - 0s 

 - 0s - loss: 0.0097 - acc: 1.0000
Epoch 339/500
 - 0s - loss: 0.0096 - acc: 1.0000
Epoch 340/500
 - 0s - loss: 0.0095 - acc: 1.0000
Epoch 341/500
 - 0s - loss: 0.0093 - acc: 1.0000
Epoch 342/500
 - 0s - loss: 0.0092 - acc: 1.0000
Epoch 343/500
 - 0s - loss: 0.0091 - acc: 1.0000
Epoch 344/500
 - 0s - loss: 0.0090 - acc: 1.0000
Epoch 345/500
 - 0s - loss: 0.0088 - acc: 1.0000
Epoch 346/500
 - 0s - loss: 0.0087 - acc: 1.0000
Epoch 347/500
 - 0s - loss: 0.0086 - acc: 1.0000
Epoch 348/500
 - 0s - loss: 0.0085 - acc: 1.0000
Epoch 349/500
 - 0s - loss: 0.0084 - acc: 1.0000
Epoch 350/500
 - 0s - loss: 0.0083 - acc: 1.0000
Epoch 351/500
 - 0s - loss: 0.0081 - acc: 1.0000
Epoch 352/500
 - 0s - loss: 0.0080 - acc: 1.0000
Epoch 353/500
 - 0s - loss: 0.0079 - acc: 1.0000
Epoch 354/500
 - 0s - loss: 0.0078 - acc: 1.0000
Epoch 355/500
 - 0s - loss: 0.0077 - acc: 1.0000
Epoch 356/500
 - 0s - loss: 0.0076 - acc: 1.0000
Epoch 357/500
 - 0s - loss: 0.0075 - acc: 1.0000
Epoch 358/500
 - 0s - loss: 0.0074

In [323]:
report()

[1. 1. 0. 0. 0.] 	 0 	 [-0.8242824] 	 [0.09073624] 	 [2.2228413 0.        0.        0.       ]
[1. 0. 1. 0. 0.] 	 1 	 [0.94265133] 	 [0.9413349] 	 [0.        2.5420465 0.        0.       ]
[1. 0. 0. 1. 0.] 	 1 	 [0.8517975] 	 [0.9904079] 	 [0.        0.        2.2970412 0.       ]
[1. 0. 0. 0. 1.] 	 0 	 [-0.99888194] 	 [0.01452958] 	 [0.        0.        0.        2.6936836]
[1. 1. 1. 0. 0.] 	 1 	 [0.88862246] 	 [0.93078804] 	 [0.       2.396347 0.       0.      ]
[1. 1. 0. 1. 0.] 	 1 	 [0.7243206] 	 [0.98010993] 	 [0.        0.        1.9532744 0.       ]
[1. 1. 0. 0. 1.] 	 0 	 [-0.99944013] 	 [0.00130149] 	 [2.6951888 0.        0.        2.6951888]
[1. 0. 1. 1. 0.] 	 1 	 [0.99909127] 	 [0.99984276] 	 [0.        2.6942482 2.6942482 0.       ]
[1. 0. 1. 0. 1.] 	 0 	 [-0.8207132] 	 [0.02876852] 	 [0.        0.        0.        2.2132163]
[1. 0. 0. 1. 1.] 	 0 	 [-0.9299016] 	 [0.01894983] 	 [0.        0.        0.        2.5076644]
[1. 1. 1. 1. 0.] 	 1 	 [0.9981852] 	 [0.99984145] 	 [0. 

In [ ]:
def np_present_by_doc(X, human_terms, index, weight):
    np_vector = np.zeros([X.shape[0],])
    
    for i, x in enumerate(X):
        if x[index] == 1:
            np_vector[i] = np.sign(weight)
    
    return np_vector

In [339]:
def load_unigrams(path, X, y):
    word_list = []
    connotation = {}
    
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            word_list.append(line.strip())
            
    for word in word_list:
        pos_count = 0
        neg_count = 0
        for i, doc in enumerate(X):
            if word in doc.lower():
                if (y[i] == 1):
                    pos_count += 1
                else:
                    neg_count += 1
                    
        if pos_count > neg_count:
            connotation[word] = 1
        else:
            connotation[word] = 0
    
    return word_list, connotation

def generate_appearance(X_train_corpus, X_test_corpus, word_list, connotation):
    y_train_agreement = []
    for i in range(len(X_train_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_train_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_train_agreement.append(doc_agreement)
        
    y_test_agreement = []
    for i in range(len(X_test_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_test_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_test_agreement.append(doc_agreement)
        
    return np.array(y_train_agreement), np.array(y_test_agreement)

# 'imdb-unigrams.txt'

### Now try on IMDB data

In [324]:
# load data
# make sure that the first shape is the IMDB training data. 

def open_pickle(path):
    import pickle
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train_original = open_pickle('../data/imdb/imdb_original_preprocessed_xtrain.pickle')
X_test_original = open_pickle('../data/imdb/imdb_original_preprocessed_xtest.pickle')
y_train_original = open_pickle('../data/imdb/imdb_original_preprocessed_ytrain.pickle')
y_test_original = open_pickle('../data/imdb/imdb_original_preprocessed_ytest.pickle')

In [342]:
# Count vectorizer 

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df = 100)
X_train = cv.fit_transform(X_train_original)
X_test = cv.transform(X_test_original)

In [340]:
word_list, connotation = load_unigrams('./imdb-unigrams.txt', X_train_original, y_train_original)

In [341]:
y_train_agreement, y_test_agreement = generate_agreements(X_train_original, X_test_original, 
                                                          y_train_original, y_test_original, 
                                                          word_list, connotation)

In [336]:
y_train_agreement[:,1]

array([0, 0, 0, ..., 0, 0, 0])

In [338]:
len(word_list)

83

In [343]:
# build the combined model
# Combined model
human_terms_len = len(word_list)

base_model = build_base_model(X_train.shape[1])

combined_input_layer = Input(shape=(X_train.shape[1],))

# build the hard coded weight for human terms
ht_input_layer = Input(shape=(human_terms_len,))

split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

# get the document prediction
label_layer = base_model(combined_input_layer)

# multiply and pass it into relu
# initialize relu layer

relu_layer = Dense(1, activation='relu', name='relu_layer', use_bias=False, kernel_initializer='ones')

# stack the multiply layer
dense_layer = []
for i in range(human_terms_len):
    dense_layer.append(relu_layer(Multiply()([split[i], label_layer])))

# concat all the result   
concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

# pass it to sigmoid layer
output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_255 (InputLayer)       (None, 3641)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3642      
Total params: 3,642
Trainable params: 3,642
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_257 (InputLayer)          (None, 83)           0                                            
__________________________________________________________________________________________________
input_256 (InputLayer)          (None, 3641)         0                                            
_________________________________________

In [350]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [351]:
y_train_tanh = y_train_original
y_train_tanh[y_train_tanh == 0] = -1

In [ ]:
combin